<div style="background-color:#000;"><img src="pqn.png"></img></div>

We import libraries for downloading financial data, running portfolio simulations, calculating asset analytics, and displaying results. We also set chart and warning preferences so output is clear and easy to follow.

## Imports and setup

In [ ]:
import os
import warnings
from datetime import datetime
import riskfolio as rp

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
from vectorbt.portfolio.enums import Direction, SizeType
from vectorbt.portfolio.nb import order_nb, sort_call_seq_nb

In [ ]:
vbt.settings.returns["year_freq"] = "252 days"

In [ ]:
warnings.filterwarnings("ignore")

Here we bring in everything we’ll need for the rest of the code. We import libraries for finance and data handling, set how return calculations treat a year, and turn off less helpful warning messages to keep our results focused.

We define which stocks and assets we want to analyze and download their historical price data from Yahoo Finance. We pull the daily closing prices for each ticker from 2010 to mid-2024.

In [ ]:
tickers = [
"UUUU", "UEC", "DNN", "URG", "tsla", "CCJ"
]

In [ ]:
data = yf.download(tickers, start="2010-01-01", end="2024-06-30", auto_adjust=False)["Close"]

We pick a mix of uranium stocks and Tesla, then pull in their daily closing prices for over a decade. This gives us the data foundation for all later calculations and analysis. By storing prices for this full window, we set up plenty of history for our simulation and optimization steps.

## Build portfolio optimization simulation

We set up a few core parameters for our backtest, including how often we'll run simulations and how many tests to run when searching for good portfolios.

In [ ]:
num_tests = 2000
ann_factor = data.vbt.returns(freq="D").ann_factor

In [ ]:
def pre_sim_func_nb(sc, every_nth):
    # Define rebalancing days
    sc.segment_mask[:, :] = False
    sc.segment_mask[every_nth::every_nth, :] = True
    return ()

This first function prepares the simulation by deciding which days we'll rebalance our portfolio. Setting a regular interval, like every 30 trading days, helps us simulate how these strategies perform with consistent rebalancing. This approach mimics what we'd do in the real world, instead of adjusting positions daily. 

In [ ]:
def pre_segment_func_nb(
    sc, find_weights_nb, history_len, ann_factor, num_tests, srb_sharpe
):
    if history_len == -1:
        # Look back at the entire time period
        close = sc.close[: sc.i, sc.from_col : sc.to_col]
    else:
        # Look back at a fixed time period
        if sc.i - history_len <= 0:
            return (np.full(sc.group_len, np.nan),)  # insufficient data
        close = sc.close[sc.i - history_len : sc.i, sc.from_col : sc.to_col]

    # Find optimal weights
    best_sharpe_ratio, weights = find_weights_nb(sc, close, num_tests)
    srb_sharpe[sc.i] = best_sharpe_ratio

    # Update valuation price and reorder orders
    size_type = np.full(sc.group_len, SizeType.TargetPercent)
    direction = np.full(sc.group_len, Direction.LongOnly)
    temp_float_arr = np.empty(sc.group_len, dtype=np.float_)
    for k in range(sc.group_len):
        col = sc.from_col + k
        sc.last_val_price[col] = sc.close[sc.i, col]
    sort_call_seq_nb(sc, weights, size_type, direction, temp_float_arr)

    return (weights,)

This function figures out the best way to split our money among assets each time we rebalance. It can use all available history or just a fixed window, depending on how much data we want. It then calculates weights targeting higher risk-adjusted returns, and updates info the simulation needs to place our orders for the new rebalance.

In [ ]:
def order_func_nb(oc, weights):
    col_i = oc.call_seq_now[oc.call_idx]
    return order_nb(
        weights[col_i],
        oc.close[oc.i, oc.col],
        size_type=SizeType.TargetPercent,
    )

Here we place our simulated orders to adjust the holdings in the portfolio. We use the weights calculated above to tell the simulation what percent to hold in each asset after each rebalance. This function ensures that, as we move through the simulation, allocations actually change according to what our optimization recommends.

In [ ]:
# This function runs the portfolio optimizer. It uses the most recent data to calculate asset returns, measures the risk of each, and computes the mix that should deliver the highest return per unit of downside risk.
def opt_weights(sc, close, num_tests):
    close = pd.DataFrame(close, columns=tickers)
    returns = close.pct_change().dropna()
    port = rp.Portfolio(returns=returns)
    port.assets_stats(method_mu="hist", method_cov="hist")
    w = port.optimization(model="Classic", rm="CDaR", obj="Sharpe", hist=True)
    weights = np.ravel(w.to_numpy())
    shp = rp.Sharpe(w, port.mu, cov=port.cov, returns=returns, rm="CDaR", alpha=0.05)
    return shp, weights

This function takes in a window of historical prices, turns them into returns, and feeds them into a classic mean-variance optimizer library. It finds weights that maximize the Sharpe ratio (risk-adjusted return) while focusing on downside risk, reflecting a cautious but practical investor mindset. This gives us a data-driven way to select allocations over time.

## Run backtest and display results

We set up the containers for our risk-adjusted return calculations, then run our entire backtest: simulating rebalancing the portfolio with optimized weights over the full period, and optionally benchmarking performance.

In [ ]:
sharpe = np.full(data.shape, np.nan)
pf = vbt.Portfolio.from_order_func(
    data,
    order_func_nb,
    pre_sim_func_nb=pre_sim_func_nb,
    pre_sim_args=(30,),
    pre_segment_func_nb=pre_segment_func_nb,
    pre_segment_args=(opt_weights, 252 * 4, ann_factor, num_tests, sharpe),
    cash_sharing=True,
    group_by=True,
    use_numba=False,
    freq="D"
)

In [ ]:
pf.plot_cum_returns()

In [ ]:
pf.stats()

In [ ]:
bm_returns = pf.benchmark_returns()
bm_returns_acc = bm_returns.vbt.returns(
    freq="1d",
    year_freq="252 days",
)
print(f"Benchmark sharpe: {bm_returns_acc.sharpe_ratio()}")
print(f"Benchmark drawdown: {bm_returns_acc.max_drawdown()}")

We initialize a grid to save Sharpe ratios, then run our custom portfolio simulation. Using our optimizer at every rebalance step, we automatically shift allocations to wherever recent data suggests risk-adjusted returns will be best. Finally, we create a visual of total returns over time and print out key stats including the standard performance measures. This makes it easy to see how our strategy compares to simply buying and holding everything.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.